In [1]:
import sys
sys.executable

'C:\\Users\\goodws\\Miniconda3\\python.exe'

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.offline import plot

from numba import jit
import time
import random

In [3]:
# dimensions of chicken
meatX = 10   # right = positive
meatY = 10   # up = positive
meatZ = 20   # out of screen = positive

# dimensions of slap
slapX = 3
slapY = 3
slapZ = 3

# dimensions of room
lenX = meatX + 2
lenY = meatY + 2
lenZ = meatZ + 2

# initial temperatures
temp_room = 20
temp_chicken = 20
#slap_temps = [30,40,50,75,100,150,200,250,300,400,500,600,750,800,1000,1500,2000,2500,3000]
slap_temps = [200,250,300,400,500,600,750,800,1000,1500,2000,2500,3000]

# temperature at which chicken is cooked
temp_cooked = 72

# parameters of heat equation
C = 1840    # J/(kg*C)
dens = 1150 # kg/m^3
K = 0.476   # W/(m*C)

# step sizes
'''pick values such that all etas <= 0.5'''
'''K_c/(C_c*dens_c) = 2.25e-7'''
'''K_c/(C_c*dens_c) = 1.23e-3'''
dx = 0.1  # 0.1
dy = 0.1
dz = 0.1
dt = 0.1
dm = dens*dx*dy*dz

h = 100                       # W/(m^2 K)
A_corner = dx*dy+dy*dz+dx*dz  # m^2
A_edge = 2*dx*dx              # m^2
A_surface = dx*dx

# combine parameters of heat equation
etaX = (K*dt)/(C*dens*(dx**2))
etaY = (K*dt)/(C*dens*(dy**2))
etaZ = (K*dt)/(C*dens*(dz**2))

cool_corner = (h*A_corner*dt)/(dm*C)
cool_edge = (h*A_edge*dt)/(dm*C)
cool_surface = (h*A_surface*dt)/(dm*C)

# boundaries of slap
#slap_start_X = int((meatX+2-slapX)/2)
#slap_end_X = slap_start_X + slapX
#slap_start_Y = int((meatY+2)/2)
#slap_end_Y = slap_start_Y + slapY
#slap_start_Z = 1
#slap_end_Z = slap_start_Z + slapZ

#print("Slap boundaries:")
#print((slap_start_X,slap_start_Y,slap_start_Z))
#print((slap_end_X,slap_end_Y,slap_end_Z))

# different number of iterations inbetween slaps -> different slap frequencies
#iter_periods = np.array([2,3,5,10,20,25,30,40,50,75])
iter_periods = np.array([80,85,90,95,100])

print('Chicken etas:',(etaX,etaY,etaZ))
print('corner',cool_corner)
print('edge',cool_edge)
print('surface',cool_surface)

Chicken etas: (2.2495274102079393e-06, 2.2495274102079393e-06, 2.2495274102079393e-06)
corner 0.00014177693761814744
edge 9.45179584120983e-05
surface 4.725897920604915e-05


In [4]:
def init_chicken(frame):
    for i in range(1,meatX+1):
        for j in range(1,meatY+1):
            for k in range(1,meatZ+1):
                frame[i,j,k] = temp_chicken
    return frame

def slap(frame,temp):
    face = random.randint(0,5)
    slap_start_X = random.randint(1,meatX+1-slapX)
    slap_end_X = slap_start_X+slapX
    slap_start_Y = random.randint(1,meatY+1-slapY)
    slap_end_Y = slap_start_Y+slapY
    slap_start_Z = random.randint(1,meatZ+1-slapZ)
    slap_end_Z = slap_start_Z+slapZ
    if face == 0:
        # bottom face
        for i in range(slap_start_X,slap_end_X):
            for k in range(slap_start_Z,slap_end_Z):
                frame[i,1,k] += temp
    elif face == 1:
        # left face
        for j in range(slap_start_Y,slap_end_Y):
            for k in range(slap_start_Z,slap_end_Z):
                frame[1,j,k] += temp
    elif face == 2:
        # top face
        for i in range(slap_start_X,slap_end_X):
            for k in range(slap_start_Z,slap_end_Z):
                frame[i,meatY,k] += temp
    elif face == 3:
        # right face
        for j in range(slap_start_Y,slap_end_Y):
            for k in range(slap_start_Z,slap_end_Z):
                frame[meatX,j,k] += temp
    elif face == 4:
        # back face
        slap_start_Z = meatZ
        slap_end_Z = meatZ
        for i in range(slap_start_X,slap_end_X):
            for j in range(slap_start_Y,slap_end_Y):
                frame[i,j,meatZ] += temp
    elif face == 5:
        # front face
        for i in range(slap_start_X,slap_end_X):
            for j in range(slap_start_Y,slap_end_Y):
                frame[i,j,1] += temp
    return frame
    '''
    for i in range(slap_start_X,slap_end_X):
        for j in range(slap_start_Y,slap_end_Y):
            frame[i,j,0] += temp_slap
    for i in range(slap_start_X,slap_end_Y):
        for k in range(slap_start_Z,slap_end_Z):
            frame[i,meatY-1,k] += temp_slap
    return frame
    '''

def is_cooked(frame):
    for i in range(1,meatX+1):
        for j in range(1,meatY+1):
            for k in range(1,meatZ+1):
                if frame[i,j,k] < temp_cooked:
                    return False
    return True

def std_dev(frame):
    sum_temps = 0
    num_temps = 0
    for i in range(1,meatX+1):
        for j in range(1,meatY+1):
            for k in range(1,meatZ+1):
                sum_temps += frame[i,j,k]
                num_temps += 1
    mean = sum_temps/num_temps
    
    var_num = 0
    for i in range(1,meatX+1):
        for j in range(1,meatY+1):
            for k in range(1,meatZ+1):
                var_num += (frame[i,j,k] - mean)**2
    std_dev = np.sqrt(var_num/num_temps)
    return std_dev

@jit(nopython=True)
def in_corner(i,j,k):
    if i == 1 and j == 1 and k == 1:
        return True
    if i == 1 and j == 1 and k == meatZ:
        return True
    if i == 1 and j == meatY and k == 1:
        return True
    if i == 1 and j == meatY and k == meatZ:
        return True
    if i == meatX and j == 1 and k == 1:
        return True
    if i == meatX and j == 1 and k == meatZ:
        return True
    if i == meatX and j == meatY and k == 1:
        return True
    if i == meatX and j == meatY and k == meatZ:
        return True
    return False

@jit(nopython=True)
def on_edge(i,j,k):
    if i == 1 and j == 1:
        return True
    if i == 1 and j == meatY:
        return True
    if i == 1 and k == 1:
        return True
    if i == 1 and k == meatZ:
        return True
    if i == meatX and j == 1:
        return True
    if i == meatX and j == meatY:
        return True
    if i == meatX and k == 1:
        return True
    if i == meatX and k == meatZ:
        return True
    if j == 1 and k == 1:
        return True
    if j == 1 and k == meatZ:
        return True
    if j == meatY and k == 1:
        return True
    if j == meatY and k == meatZ:
        return True
    return False

@jit(nopython=True)
def on_surface(i,j,k):
    if i == 1 or i == meatX or j == 1 or j == meatY or k == 1 or k == meatZ:
        return True
    else:
        return False
        

@jit(nopython=True,parallel=True)
def sim_step(old_frame,new_frame):
    for i in range(1,meatX+1):
        for j in range(1,meatY+1):
            for k in range(1,meatZ+1):

                x_comp = etaX*(old_frame[i+1,j,k] + old_frame[i-1,j,k] - 2*old_frame[i,j,k])
                y_comp = etaY*(old_frame[i,j+1,k] + old_frame[i,j-1,k] - 2*old_frame[i,j,k])
                z_comp = etaZ*(old_frame[i,j,k+1] + old_frame[i,j,k-1] - 2*old_frame[i,j,k])
                
                new_frame[i,j,k] = old_frame[i,j,k] + x_comp + y_comp + z_comp
                
                # cooling term for parts of chicken on surface
                if in_corner(i,j,k):
                    new_frame[i,j,k] -= cool_corner*(old_frame[i,j,k] - temp_room)
                elif on_edge(i,j,k):
                    new_frame[i,j,k] -= cool_edge*(old_frame[i,j,k] - temp_room)
                elif on_surface(i,j,k):
                    new_frame[i,j,k] -= cool_surface*(old_frame[i,j,k] - temp_room)
                # debugging
                #if new_frame[i,j,k] < temp_room:
                    #print("Invalid temp")
                    #print((i,j,k),(x_comp,y_comp,z_comp),new_frame[i,j,k])
                    #for a in range(-1,2):
                        #print((i+a,j,k), old_frame[i+a,j,k])
                    #break          
    return new_frame

def temp_bounds(frame):
    min_temp = 2*temp_cooked
    min_x = 0
    min_y = 0
    min_z = 0
    max_temp = 0
    max_x = 0
    max_y = 0
    max_z = 0
    for i in range(1,meatX+1):
        for j in range(1,meatY+1):
            for k in range(1,meatZ+1):
                if frame[i,j,k] < min_temp:
                    min_temp = frame[i,j,k]
                    min_x = i
                    min_y = j
                    min_z = k
                if frame[i,j,k] > max_temp:
                    max_temp = frame[i,j,k]
                    max_x = i
                    max_y = j
                    max_z = k
    #return (min_temp,max_temp)
    return ((min_temp,min_x,min_y,min_z),(max_temp,max_x,max_y,max_z))

def print_results(frame,freq,temp,time,sd):
    print("Finished cooking a chicken after",time,"iterations")
    print("\tslap temperature:",temp)
    print("\titers per slap:",freq)
    bounds = temp_bounds(frame)
    print("\tmin temp is",bounds[0][0], "at", (bounds[0][1],bounds[0][2],bounds[0][3]))
    print("\tmax temp is",bounds[1][0], "at", (bounds[1][1],bounds[1][2],bounds[1][3]))
    print("\tstd dev of temp is",sd)
    print()

In [5]:
# run the simulation
chickens = []
results = []

freq_edge = []
temp_edge = []

for t in slap_temps:
    abort_mission = False
    for n in iter_periods:
        chicken = np.empty((lenX,lenY,lenZ))
        chicken.fill(temp_room)
        chicken = init_chicken(chicken)
        time_step = 0
        old_min_temp = temp_chicken
        while ((is_cooked(chicken) == False) and abort_mission == False):
            old_frame = np.copy(chicken)
            new_frame = np.copy(chicken)
            if time_step % n == 0:
                old_frame = slap(old_frame,t)
            chicken = sim_step(old_frame,new_frame)
            time_step += 1
            if time_step % 100000 == 0:
                #print(time_step, ":", temp_bounds(chicken))
                new_min_temp = np.amin(chicken)
                if new_min_temp < old_min_temp or time_step > 100000000000:
                    print("Chicken is not cooking. Aborting mission.")
                    print("\tslap temp",t)
                    print("\titers per slap",n)
                    print("time",time_step)
                    print(temp_bounds(chicken))
                    print()
                    freq_edge.append(1/(n*dt))
                    temp_edge.append(t)
                    abort_mission = True
        if abort_mission == False:
            sd = std_dev(chicken)
            print_results(chicken,n,t,time_step,sd)
            chickens.append(chicken)
            results.append([time_step,t,n,sd])

KeyboardInterrupt: 

In [ ]:
# Plot showing boundary of conditions at which chicken can/cannot be cooked
plt.plot(temp_edge,freq_edge)
plt.title("Cooking Boundary")
plt.xlabel("Slap Temperature [degrees Celsius]")
plt.ylabel("Slap Frequency [slaps/second]")
plt.show()

In [ ]:
iters = []
temp = []
ips = []
sd = []

dt = 0.1

for i in range(0,len(results)):
    #if i == 4 or i == 20 or i == 26 or i == 39 or i == 46:
        #continue
    iters.append(results[i][0])
    temp.append(results[i][1])
    ips.append(results[i][2])
    sd.append(results[i][3])

times = []
for i in range(0,len(iters)):
    times.append(iters[i]*dt)

freq = []
for i in range(0,len(ips)):
    freq.append(1/(ips[i]*dt))

fig1 = plt.figure(1)
fig1.set_figheight(10)
fig1.set_figwidth(10)
ax1 = fig1.add_subplot(111,projection='3d')
ax1.plot_trisurf(temp,freq,sd)
ax1.set_title("Eveness of Temperature")
ax1.set_xlabel("Slap Temperature [degrees Celsius]")
ax1.set_ylabel("Slap Frequency [slaps/second]")
ax1.set_zlabel("Standard Deviation of Temperature")

In [ ]:
fig2 = plt.figure(2)
fig2.set_figheight(10)
fig2.set_figwidth(10)
ax2 = fig2.add_subplot(111,projection='3d')
ax2.plot_trisurf(temp,freq,times)
ax2.set_title("Time Needed to Cook Chicken")
ax2.set_xlabel("Slap Temperature [degrees Celsius]")
ax2.set_ylabel("Slap Frequency [slaps/second]")
ax2.set_zlabel("Cooking Time [seconds]")

In [ ]:
trace1 = go.Scatter3d(
    x=temp,
    y=freq,
    z=sd,
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )

)

data = [trace1]
layout = go.Layout(
    title = 'A Comparison of Cooking Parameters',
    scene = Scene(
        xaxis=XAxis(title='Slap Temperature [degrees Celsius]'),
        yaxis=YAxis(title='Slap Frequency [slaps/second]'),
        zaxis=ZAxis(title='Standard Deviation of Temperature')
    ),

    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
plot(fig, filename='./sd_graph.html')

In [ ]:
trace1 = go.Scatter3d(
    x=temp,
    y=freq,
    z=times,
    mode='markers',
    marker=dict(
        size=12,
        color='rgba(255,69,0,0.84)',
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )

)

data = [trace1]
layout = go.Layout(
    title = 'A Comparison of Cooking Parameters',
    scene = Scene(
        xaxis=XAxis(title='Slap Temperature [degrees Celsius]'),
        yaxis=YAxis(title='Slap Frequency [slaps/second]'),#yaxis=YAxis(title='One slap every y iterations'),
        zaxis=ZAxis(title='Cooking Time [seconds]')
    ),

    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
plot(fig, filename='./time_graph.html')

In [ ]:
# most evenly-cooked chicken
even_pos = 0
min_sd = results[0][3]
for i in range(1,len(sd)):
    if results[i][3] < min_sd:
        min_sd = results[i][3]
        even_pos = i

print(results[even_pos])
print("Time:", times[even_pos], "seconds")
print("Slap Temp:", temp[even_pos], "degrees Celsius")
print("Slap Freq:", freq[even_pos], "slaps/second")
print("Std Dev:", sd[even_pos])

In [ ]:
# fastest-cooked chicken
time_pos = 0
min_time = times[0]
for i in range(1,len(times)):
    if times[i] < min_time:
        min_time = times[i]
        time_pos = i

print(results[time_pos])
print("Time:", times[time_pos], "seconds")
print("Slap Temp:", temp[time_pos], "degrees Celsius")
print("Slap Freq:", freq[time_pos], "slaps/second")
print("Std Dev:", sd[time_pos])

In [ ]:
max_temp = 920
# heatmaps of most evenly-cooked chicken

colorinterpolation = 100
colourMap = plt.cm.jet

# front face

X_front, Y_front = np.meshgrid(dx*np.arange(0,lenX,1), dy*np.arange(0,lenY,1))
sol_front = chickens[even_pos][:,:,1]
sol_front[sol_front == np.inf] = np.nan

plt.title("Most-Evenly Cooked: Front Face")
plt.contourf(X_front, Y_front, sol_front, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_front), vmax = np.nanmax(sol_front))
plt.xlabel('x-position [m]')
plt.ylabel('y-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# left face

Z_left, Y_left = np.meshgrid(dz*np.arange(0,lenZ,1), dy*np.arange(0,lenY,1))
sol_left = chickens[even_pos][1,:,:]
sol_left[sol_left == np.inf] = np.nan

plt.title("Most-Evenly Cooked: Left Face")
plt.contourf(Z_left, Y_left, sol_left, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_left), vmax = np.nanmax(sol_left))
plt.xlabel('z-position [m]')
plt.ylabel('y-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# right face

Z_right, Y_right = np.meshgrid(dz*np.arange(0,lenZ,1), dy*np.arange(0,lenY,1))
sol_right = chickens[even_pos][meatX,:,:]
sol_right[sol_right == np.inf] = np.nan

plt.title("Most-Evenly Cooked: Right Face")
plt.contourf(Z_right, Y_right, sol_right, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_right), vmax = np.nanmax(sol_right))
plt.xlabel('z-position [m]')
plt.ylabel('y-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# top face

Z_top, X_top = np.meshgrid(dz*np.arange(0,lenZ,1), dx*np.arange(0,lenX,1))
sol_top = chickens[even_pos][:,meatY,:]
sol_top[sol_top == np.inf] = np.nan

plt.title("Most-Evenly Cooked: Top Face")
plt.contourf(Z_top, X_top, sol_top, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_top), vmax = np.nanmax(sol_top))
plt.xlabel('z-position [m]')
plt.ylabel('x-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# bottom face

Z_bottom, X_bottom = np.meshgrid(dz*np.arange(0,lenZ,1), dx*np.arange(0,lenX,1))
sol_bottom = chickens[even_pos][:,1,:]
sol_bottom[sol_bottom == np.inf] = np.nan

plt.title("Most-Evenly Cooked: Bottom Face")
plt.contourf(Z_bottom, X_bottom, sol_bottom, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_bottom), vmax = np.nanmax(sol_bottom))
plt.xlabel('z-position [m]')
plt.ylabel('x-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# back face

X_back, Y_back = np.meshgrid(dx*np.arange(0,lenX,1), dy*np.arange(0,lenY,1))
sol_back = chickens[even_pos][:,:,meatZ]
sol_back[sol_back == np.inf] = np.nan

plt.title("Most-Evenly Cooked: Back Face")
plt.contourf(X_back, Y_back, sol_back, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_back), vmax = np.nanmax(sol_back))
plt.xlabel('x-position [m]')
plt.ylabel('y-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# cross section

X_cross, Y_cross = np.meshgrid(dx*np.arange(0,lenX,1), dy*np.arange(0,lenY,1))
sol_cross = chickens[even_pos][:,:,int(meatZ/2)]
sol_cross[sol_cross == np.inf] = np.nan

plt.title("Most-Evenly Cooked: Cross-Section")
plt.contourf(X_cross, Y_cross, sol_cross, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_cross), vmax = np.nanmax(sol_cross))
plt.xlabel('x-position [m]')
plt.ylabel('y-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

In [ ]:
max_temp = 720000
# heatmaps of fastest-cooked chicken

#colorinterpolation = 100
#colourMap = plt.cm.jet

# front face

X_front, Y_front = np.meshgrid(dx*np.arange(0,lenX,1), dy*np.arange(0,lenY,1))
sol_front = chickens[time_pos][:,:,1]
sol_front[sol_front == np.inf] = np.nan

plt.title("Most-Quickly Cooked: Front Face")
plt.contourf(X_front, Y_front, sol_front, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_front), vmax = np.nanmax(sol_front))
plt.xlabel('x-position [m]')
plt.ylabel('y-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# left face

Z_left, Y_left = np.meshgrid(dz*np.arange(0,lenZ,1), dy*np.arange(0,lenY,1))
sol_left = chickens[time_pos][1,:,:]
sol_left[sol_left == np.inf] = np.nan

plt.title("Most-Quickly Cooked: Left Face")
plt.contourf(Z_left, Y_left, sol_left, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_left), vmax = np.nanmax(sol_left))
plt.xlabel('z-position [m]')
plt.ylabel('y-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# right face

Z_right, Y_right = np.meshgrid(dz*np.arange(0,lenZ,1), dy*np.arange(0,lenY,1))
sol_right = chickens[time_pos][meatX,:,:]
sol_right[sol_right == np.inf] = np.nan

plt.title("Most-Quickly Cooked: Right Face")
plt.contourf(Z_right, Y_right, sol_right, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_right), vmax = np.nanmax(sol_right))
plt.xlabel('y-position [m]')
plt.ylabel('z-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# top face

Z_top, X_top = np.meshgrid(dz*np.arange(0,lenZ,1), dx*np.arange(0,lenX,1))
sol_top = chickens[time_pos][:,meatY,:]
sol_top[sol_top == np.inf] = np.nan

plt.title("Most-Quickly Cooked: Top Face")
plt.contourf(Z_top, X_top, sol_top, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_top), vmax = np.nanmax(sol_top))
plt.xlabel('z-position [m]')
plt.ylabel('x-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# bottom face

Z_bottom, X_bottom = np.meshgrid(dz*np.arange(0,lenZ,1), dx*np.arange(0,lenX,1))
sol_bottom = chickens[time_pos][:,1,:]
sol_bottom[sol_bottom == np.inf] = np.nan

plt.title("Most-Quickly Cooked: Bottom Face")
plt.contourf(Z_bottom, X_bottom, sol_bottom, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_bottom), vmax = np.nanmax(sol_bottom))
plt.xlabel('z-position [m]')
plt.ylabel('x-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# back face

X_back, Y_back = np.meshgrid(dx*np.arange(0,lenX,1), dy*np.arange(0,lenY,1))
sol_back = chickens[time_pos][:,:,meatZ]
sol_back[sol_back == np.inf] = np.nan

plt.title("Most-Quickly Cooked: Back Face")
plt.contourf(X_back, Y_back, sol_back, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_back), vmax = np.nanmax(sol_back))
plt.xlabel('x-position [m]')
plt.ylabel('y-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

# cross section

X_cross, Y_cross = np.meshgrid(dx*np.arange(0,lenX,1), dy*np.arange(0,lenY,1))
sol_cross = chickens[time_pos][:,:,int(meatZ/2)]
sol_cross[sol_cross == np.inf] = np.nan

plt.title("Most-Quickly Cooked: Cross-Section")
plt.contourf(X_cross, Y_cross, sol_cross, colorinterpolation, cmap=colourMap, vmin = np.nanmin(sol_cross), vmax = np.nanmax(sol_cross))
plt.xlabel('x-position [m]')
plt.ylabel('y-position [m]')
plt.colorbar().set_label("Temperature [degrees Celsius]")
plt.clim(temp_room,max_temp)
plt.axis('scaled')
plt.show()

In [ ]:
'''
# find chicken with maximum taste
max_taste_pos = 0
max_taste = tastes[0]
for i in range(0,len(tastes)):
    if tastes[i] > max_taste:
        max_taste_pos = i
        max_taste = tastes[i]

# find minimum and maximum temperature points within tastiest chicken
best_chicken = chickens[max_taste_pos]
min_temp = best_chicken[0,0,0]
max_temp = best_chicken[0,0,0]
for i in range(0,meatX):
    for j in range(0,meatY):
        for k in range(0,meatZ):
            if best_chicken[i,j,k] < min_temp:
                min_temp = best_chicken[i,j,k]
            elif best_chicken[i,j,k] > max_temp:
                max_temp = best_chicken[i,j,k]

# print the results
print("Tastiest chicken cooked with slap frequency", freq[max_taste_pos], "s^{-1}", "and slap temperature (add this code later maybe)")
print("Minimum temperature in tastiest chicken", min_temp, "degrees Celsius")
print("Maximum temperature in tastiest chicken", max_temp, "degrees Celsius")
'''

In [ ]:
'''
# heatmap of tastiest chicken and maybe worst-tasting chicken
X = np.linspace(0,meatX*dx,meatX*dx)
Y = np.linspace(0,meatY*dy,meatY*dy)
Z = np.linspace(0,meatZ*dz,meatZ*dz)

trace1 = go.Scatter3d(
    x=X,
    y=Y,
    z=Z,
    mode='markers',
    marker=dict(
        size=12,
        color=best_chicken.flatten(),                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
plot(fig, filename='./best-chicken.html')
'''